In [4]:
import pandas as pd
import plotly.express as px
from datetime import datetime

def process_csv(filepath):
    # Read the CSV file
    df = pd.read_csv(filepath)
    # Process the data as needed
    return df

In [5]:
main_df = process_csv('5_years_financial_data.csv')
main_df['Date'] = pd.to_datetime(main_df['Date'], format="%d/%m/%Y")
main_df

,Date,Category,Amount
0,2019-01-01,Transportation,84.04
1,2019-01-02,Entertainment,97.82
2,2019-01-03,Entertainment,63.70
3,2019-01-04,Dining Out,90.04
4,2019-01-05,Transportation,124.11
...,...,...,...
1821,2023-12-27,Entertainment,164.03
1822,2023-12-28,Dining Out,153.23
1823,2023-12-29,Groceries,84.57
1824,2023-12-30,Entertainment,106.17


In [6]:
main_df.dtypes

Date        datetime64[ns]
Category            object
Amount             float64
dtype: object

### Utility functions

In [7]:
# input year as int
def filter_by_year(df, year):
    """
    Filter a DataFrame by a given year, returning a new DataFrame
    with rows only from that year.

    Parameters:
    - df: A pandas DataFrame with a 'Date' column in datetime format.
    - year: An integer representing the year to filter by.

    Returns:
    - A pandas DataFrame with rows from the specified year.
    """

    # Filter the DataFrame for the given year
    filtered_df = df[df['Date'].dt.year == year]
    
    return filtered_df

# input month as int
def filter_by_month(df, month):
    """
    Filter a DataFrame by a given month name, returning a new DataFrame
    with rows only from that month.

    Parameters:
    - df: A pandas DataFrame with a 'Date' column in datetime format.
    - month: Integer representing the month to filter by, e.g., 2 for February

    Returns:
    - A pandas DataFrame with rows from the specified month.
    """
    # TODO this in javascript instead for input
    # Create a dictionary to map month names to month numbers
    # month_to_num = {
    #     'January': 1,
    #     'February': 2,
    #     'March': 3,
    #     'April': 4,
    #     'May': 5,
    #     'June': 6,
    #     'July': 7,
    #     'August': 8,
    #     'September': 9,
    #     'October': 10,
    #     'November': 11,
    #     'December': 12
    # }
    
    # Get the month number for the given month name
    # month_num = month_to_num.get(month.capitalize())
    months = [1,2,3,4,5,6,7,8,9,10,11,12]


    if month not in months:
        raise ValueError(f"Invalid month name: {month}")
    
    # Filter the DataFrame for the given month number
    filtered_df = df[df['Date'].dt.month == month]
    
    return filtered_df






In [8]:

def sum_expenditure_by_month(df):
    """
    Sum the expenditure by month, with the month and year formatted as 'January 2019', etc.

    Parameters:
    - df: pandas DataFrame with a 'Date' and 'Amount' column.

    Returns:
    - A new DataFrame with 'Month' and 'Total Expenditure' columns.
    """
    
    # Group by year and month, and sum the amounts
    df_grouped = df.groupby(df['Date'].dt.to_period("M"))['Amount'].sum().reset_index()

    # Convert the 'Date' periods back to timestamp to format them
    df_grouped['Date'] = df_grouped['Date'].dt.to_timestamp()

    # Format the 'Date' column as 'Month Year'
    df_grouped['Month'] = df_grouped['Date'].dt.strftime('%B %Y')

    # Rename columns and select only the needed ones
    result_df = df_grouped[['Month', 'Amount']].rename(columns={'Amount': 'Total Expenditure'})

    return result_df


In [9]:
def aggregate_spending_by_category(df):
    """
    Aggregate spending by category for each month.

    Parameters:
    - df: pandas DataFrame with columns 'Date', 'Category', and 'Amount'.

    Returns:
    - A new DataFrame with months as rows, categories as columns, and the sum of expenditures as values.
    """

    # Extract year and month from 'Date' and create a new column for it
    df['YearMonth'] = df['Date'].dt.to_period('M')
    
    # Group by the new 'YearMonth' column and 'Category', and sum 'Amount'
    result_df = df.groupby(['YearMonth', 'Category'])['Amount'].sum().unstack(fill_value=0)

    # Reset the index to make 'YearMonth' a column again
    result_df.reset_index(inplace=True)
    
    # Optionally format 'YearMonth' to 'Month Year' for better readability
    result_df['YearMonth'] = result_df['YearMonth'].dt.strftime('%B %Y')

    return result_df

# Example usage
# Assuming 'df' is your DataFrame loaded with the appropriate data
# summed_df = aggregate_spending_by_category(df)
# print(summed_df)


In [10]:
def aggregate_spending_by_category_long_form(df):
    """
    Aggregate spending by category for each month in a long form.

    Parameters:
    - df: pandas DataFrame with columns 'Date', 'Category', and 'Amount'.

    Returns:
    - A new DataFrame in long form with 'YearMonth', 'Category', and 'Amount'.
    """
    
    # Extract year and month from 'Date' and create a new column for it
    df['YearMonth'] = df['Date'].dt.to_period('M')
    
    # Group by the new 'YearMonth' column and 'Category', and sum 'Amount'
    result_df = df.groupby(['YearMonth', 'Category'])['Amount'].sum().reset_index()
    
    # Optionally format 'YearMonth' to 'Month Year' for better readability
    result_df['YearMonth'] = result_df['YearMonth'].dt.strftime('%B %Y')

    return result_df

### Charts test

1. pie chart: by this month, this year, select year?, (week is harder to implement, TODO later),, maybe not pie chart...
2. bar chart: breakdown 6 categories by this month, this year, select year?, (week is harder to implement, TODO later)
3. line chart: total expenses over time by month

In [11]:
# list of available years
unique_years = main_df['Date'].dt.year.unique()
unique_years

array([2019, 2020, 2021, 2022, 2023])

#### pie charts (discontinued)

In [ ]:
# TODO check if this is aggregated , summed up
# pie_df = filter_by_year(main_df, 2019)

# fig = px.pie(pie_df, values='Amount', names='Category', title='Expenditure breakdown in 2019') # make this an input, year or month
# fig.show()

In [8]:
# filter by year then month



#### bar charts

In [14]:
# to test
def barchart(df):
    """
    Aggregate spending by category for each month in a long form suitable for creating datasets
    for a stacked bar chart in Chart.js, ensuring chronological order of months.

    Parameters:
    - df: pandas DataFrame with columns 'Date', 'Category', and 'Amount'.

    Returns:
    - JSON object with 'labels' and 'datasets' suitable for Chart.js.
    """

    # Extract year and month from 'Date' and create a new column for it, converting to period
    df['YearMonth'] = df['Date'].dt.to_period('M')
    
    # Group by the new 'YearMonth' column and 'Category', and sum 'Amount'
    result_df = df.groupby(['YearMonth', 'Category'])['Amount'].sum().unstack(fill_value=0).reset_index()

    # Sort the DataFrame by the datetime period (implicitly does it chronologically)
    result_df['YearMonth'] = result_df['YearMonth'].dt.to_timestamp()  # Convert to timestamp for sorting
    result_df.sort_values('YearMonth', inplace=True)
    result_df['YearMonth'] = result_df['YearMonth'].dt.strftime('%B %Y')  # Convert back to string for display

    # Convert the DataFrame to a dictionary that fits the structure needed for Chart.js
    datasets = []
    colors = {
        'Dining Out':'#DE3163',
        'Utilities':'#808080',
        'Transportation':'#0047AB',
        'Groceries':'#CC5500',
        'Entertainment':'#C3B1E1',
        'Health':'#008000',
    }
    categories = result_df.columns[1:]  # Skip the 'YearMonth' column

    for category in categories:
        datasets.append({
            'label': category,
            'data': result_df[category].tolist(),
            'backgroundColor': colors[category]
        })

    return {
        'labels': result_df['YearMonth'].tolist(),
        'datasets': datasets
    }


In [15]:
bar_df = barchart(main_df)
bar_df

{'labels': ['January 2019',
  'February 2019',
  'March 2019',
  'April 2019',
  'May 2019',
  'June 2019',
  'July 2019',
  'August 2019',
  'September 2019',
  'October 2019',
  'November 2019',
  'December 2019',
  'January 2020',
  'February 2020',
  'March 2020',
  'April 2020',
  'May 2020',
  'June 2020',
  'July 2020',
  'August 2020',
  'September 2020',
  'October 2020',
  'November 2020',
  'December 2020',
  'January 2021',
  'February 2021',
  'March 2021',
  'April 2021',
  'May 2021',
  'June 2021',
  'July 2021',
  'August 2021',
  'September 2021',
  'October 2021',
  'November 2021',
  'December 2021',
  'January 2022',
  'February 2022',
  'March 2022',
  'April 2022',
  'May 2022',
  'June 2022',
  'July 2022',
  'August 2022',
  'September 2022',
  'October 2022',
  'November 2022',
  'December 2022',
  'January 2023',
  'February 2023',
  'March 2023',
  'April 2023',
  'May 2023',
  'June 2023',
  'July 2023',
  'August 2023',
  'September 2023',
  'October 2023'

In [23]:
# aggregate by year

bar_df = aggregate_spending_by_category_long_form(main_df)
bar_df


,YearMonth,Category,Amount
0,January 2019,Dining Out,732.17
1,January 2019,Entertainment,688.52
2,January 2019,Groceries,131.16
3,January 2019,Health,82.77
4,January 2019,Transportation,487.84
...,...,...,...
353,December 2023,Entertainment,582.84
354,December 2023,Groceries,390.44
355,December 2023,Health,635.37
356,December 2023,Transportation,626.74


In [24]:
#### stacked bar charts

fig = px.bar(bar_df, x="YearMonth", y="Amount", color="Category", title="Expenditure breakdown")
fig.show()

#### line charts

In [15]:
# aggregate total spending per month, per year
line_df = sum_expenditure_by_month(main_df)
line_df

,Month,Total Expenditure
0,January 2019,2457.76
1,February 2019,3299.60
2,March 2019,3529.34
3,April 2019,3367.66
4,May 2019,3652.59
5,June 2019,3318.51
6,July 2019,4003.08
7,August 2019,3212.92
8,September 2019,3095.22
9,October 2019,3418.11


In [17]:

fig = px.line(line_df, x='Month', y='Total Expenditure', title='Total Expenditure', markers=True) # , color='country'
fig.show()